In [187]:
#pip install seaborn
#pip install openpyxl

SyntaxError: invalid syntax (389408990.py, line 3)

In [1]:
#pip install scikit-learn

In [2]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression


**Upload DataFrames**

In [3]:
# DO NOT CHANGE THESE DATAFRAMES
# CREATE COPIES
df_threats = pd.read_csv('df_threats_ss.csv', parse_dates=True)
df_net_crime = pd.read_csv('LossFromNetCrime.csv')
df_world = pd.read_csv('world-data-2023.csv', index_col = 'Country')

#### EU Country Codes Dictionary (Hide Cell)

In [4]:
eu_country_codes = {
    "PT": "Portugal", "AT": "Austria", "RO": "Romania", "BE": "Belgium",
    "BG": "Bulgaria", "SE": "Sweden", "SI": "Slovenia", "SK": "Slovakia",
    "CY": "Cyprus", "CZ": "Czech Republic", "DE": "Germany", "DK": "Denmark",
    "EE": "Estonia", "ES": "Spain", "FI": "Finland", "FR": "France",
    "GR": "Greece", "HR": "Croatia", "HU": "Hungary", "IE": "Ireland",
    "IT": "Italy", "LT": "Lithuania", "LU": "Luxembourg", "LV": "Latvia",
    "MT": "Malta", "NL": "Netherlands", "PL": "Poland", "EU": "Europe region"
}

### Update the dataframes to include EU countries only 

In [5]:
df_threats = df_threats[df_threats['Receiver country'].isin(eu_country_codes.values())]
df_net_crime = df_net_crime[df_net_crime['Country'].isin(eu_country_codes.keys())]

# works differently for index column
valid_indices = df_world.index.intersection(eu_country_codes.values())
df_world = df_world.loc[valid_indices]

# Section 1 - Cyber Crime Losses & Complaints – Guided Analysis

Welcome! This notebook is designed for **non‑technical and non‑coding users** who need to explore a cyber crime dataset and prepare results for Power BI.  \
We will walk through each step, explain what the code does in plain language, and export tables to CSV files so you can create charts in Power BI without writing any code yourself.  \
If a plot would normally appear in Python, we instead save the underlying data for Power BI.  \
At the end you will have several CSV files ready to import into Power BI to create your own charts.

## 1. Load and prepare the data

In this section we will:

1. Import the Python packages we need.
2. Create a folder to store our output files (so everything is in one place).
3. Read the cyber crime dataset from `LossFromNetCrime.csv`.
4. Clean the data by filling missing country names and converting numeric columns to numbers.
5. Define the list of years (2019–2024) we will analyse.

Everything is commented so you can follow along even if you have never coded before.

In [6]:
# 1.1 Create an output folder if it does not already exist.
# All of our CSV files will be saved here.
os.makedirs('Code_Output', exist_ok=True)

# 1.2 Load the df_loss dataframe
df_loss = df_net_crime.copy()

# 1.3 Replace any missing country names with 'Unknown'.
# This prevents errors when sorting or grouping.
df_loss['Country'] = df_loss['Country'].fillna('Unknown')

# 1.4 Convert all columns except 'Country' to numbers.
# Sometimes numbers are stored as text; this forces them to numeric so we can add and average them.
numeric_cols = [c for c in df_loss.columns if c != 'Country']
df_loss[numeric_cols] = df_loss[numeric_cols].apply(pd.to_numeric, errors='coerce')

# 1.5 Define the years we will analyse.
years = [2019, 2020, 2021, 2022, 2023, 2024]

# 1.6 Display the first few rows to verify the data loaded correctly.
df_loss.head()

,Country,2019_Complaints,2019_Losses,2020_Complaints,2020_Losses,2021_Complaints,2021_Losses,2022_Complaints,2022_Losses,2023_Complaints,2023_Losses,2024_Complaints,2024_Losses
2,PT,1119,13870074,2020,12391290,2102,18205913,1918,30859319,2178,28700418,2209,40192274
10,AT,6942,47306368,10963,41464758,15412,116199865,11239,133170619,9614,163158856,14534,145551079
13,RO,17366,121565659,25108,152115092,31549,301879874,28823,349041126,31255,552881444,33338,534185027
18,BE,6275,61314011,10620,81868291,11842,142566044,10859,217821448,11848,184239517,12375,303148050
19,BG,14189,118129647,26712,111207806,24968,248788813,23525,338814414,26404,443783119,24208,522143452


## 2. Totals by year and changes

To understand the overall trend, we need to know how many complaints and how much money was lost across **all countries** each year.

We will:

- Sum complaints and losses for each year.
- Compute the change from one year to the next (this shows whether things are getting better or worse).
- Save the results to a CSV file for Power BI (`totals.csv`).
- Display the table in the notebook for reference.

In [7]:
# 2.1 Calculate total complaints and losses for each year across all countries
totals = []
for year in years:
    total_complaints = df_loss[f'{year}_Complaints'].sum()
    total_losses = df_loss[f'{year}_Losses'].sum()
    totals.append({
        'Year': year,
        'Total_Complaints': total_complaints,
        'Total_Losses': total_losses
    })

# 2.2 Convert the list of totals to a DataFrame
totals_df = pd.DataFrame(totals)

# 2.3 Compute year‑over‑year changes
totals_df['Complaints_Change'] = totals_df['Total_Complaints'].diff()
totals_df['Losses_Change'] = totals_df['Total_Losses'].diff()

# 2.4 Save the totals and changes to CSV for Power BI
totals_df.to_csv('Code_Output/totals.csv', index=False)

# 2.5 Display the totals table
totals_df

,Year,Total_Complaints,Total_Losses,Complaints_Change,Losses_Change
0,2019,592135,5806561305,NaN,NaN
1,2020,926298,7593895214,334163.0,1.787334e+09
2,2021,1023987,11503890534,97689.0,3.909995e+09
3,2022,991992,16708820739,-31995.0,5.204930e+09
4,2023,1112025,21206120165,120033.0,4.497299e+09
5,2024,1035712,23799224049,-76313.0,2.593104e+09


#### Plot for Testing Purposes

In [8]:
#plt.plot(totals_df['Year'], totals_df['Total_Complaints'])
#plt.plot(totals_df['Year'], totals_df['Total_Losses'])

## 3. Top countries by complaints and losses (per year)

Power BI visualisations often focus on the top performers or worst offenders. We will identify the **top 5 countries** for each year based on both complaints and losses. Instead of plotting the results here, we build tidy tables for Power BI:

- `top5_complaints_chart_data.csv`: Each row contains the year, country, and complaints (in millions).
- `top5_losses_chart_data.csv`: Each row contains the year, country, and losses (in billions).

These files allow you to create bar charts in Power BI showing the top countries year by year.

In [9]:
# 3.1 Build the data for top 5 complaints per year
complaint_rows = []
for year in years:
    col_name = f'{year}_Complaints'
    # Sort the countries by complaints for this year and take the top 5
    top5 = df_loss.sort_values(by=col_name, ascending=False).head(5)
    for country, value in zip(top5['Country'], top5[col_name]):
        complaint_rows.append({
            'Year': year,
            'Country': country,
            'Complaints_Millions': value / 1_000_000
        })

complaint_chart_df = pd.DataFrame(complaint_rows)

# 3.2 Save the data to CSV
complaint_chart_df.to_csv('Code_Output/top5_complaints_chart_data.csv', index=False)

# 3.3 Display the first few rows for reference
complaint_chart_df.head()

,Year,Country,Complaints_Millions
0,2019,FR,0.160810
1,2019,DE,0.095136
2,2019,NL,0.049692
3,2019,SE,0.043233
4,2019,IT,0.042362


#### Visual Table (Hide Cell)

In [10]:
table = pd.DataFrame()

table['2019'] = list(complaint_chart_df[complaint_chart_df['Year'] == 2019]['Country'])
table['2020'] = list(complaint_chart_df[complaint_chart_df['Year'] == 2020]['Country'])
table['2021'] = list(complaint_chart_df[complaint_chart_df['Year'] == 2021]['Country'])
table['2022'] = list(complaint_chart_df[complaint_chart_df['Year'] == 2022]['Country'])
table['2023'] = list(complaint_chart_df[complaint_chart_df['Year'] == 2023]['Country'])
table['2024'] = list(complaint_chart_df[complaint_chart_df['Year'] == 2024]['Country'])

complaint_chart_df

table

,2019,2020,2021,2022,2023,2024
0,FR,FR,FR,DE,FR,FR
1,DE,DE,DE,FR,DE,DE
2,NL,NL,IT,SE,IT,NL
3,SE,IT,NL,NL,NL,IT
4,IT,SE,SE,IT,SE,ES


In [11]:
# 3.4 Build the data for top 5 losses per year
loss_rows = []
for year in years:
    col_name = f'{year}_Losses'
    top5 = df_loss.sort_values(by=col_name, ascending=False).head(5)
    for country, value in zip(top5['Country'], top5[col_name]):
        loss_rows.append({
            'Year': year,
            'Country': country,
            'Losses_Billions': value / 1_000_000_000
        })

loss_chart_df = pd.DataFrame(loss_rows)

# 3.5 Save to CSV
loss_chart_df.to_csv('Code_Output/top5_losses_chart_data.csv', index=False)

# 3.6 Display the first few rows
loss_chart_df.head()

,Year,Country,Losses_Billions
0,2019,DE,1.559416
1,2019,FR,0.967759
2,2019,NL,0.572305
3,2019,PL,0.390366
4,2019,ES,0.334886


## 5. Forecasting 2025 using trained linear regression

This code trains separate linear models on the 2019‑2024 data for each selected country to predict both losses and complaints for 2025, includes a Year column in the output, and builds a tidy table of actual vs. predicted values

What this does:

Fits a straight-line model to each country’s 2019–2024 losses and complaints.

Predicts losses and complaints only for 2025 (no extrapolation beyond 2025).

Outputs predicted_losses_linear_2025.csv with columns Country, Year, Predicted_Losses, and Predicted_Complaints.

Creates actual_vs_predicted_linear_by_country.csv with a row for every (country, year) pair, including actual 2019–2024 values and the 2025 prediction, plus a Type column (Actual/Predicted). This tidy file is ideal for a Power BI line chart: set Year on the x-axis (categorical), Losses_Billions and/or Complaints_Millions as values (Sum), Country as legend, and Type to distinguish solid vs. dotted segments.

In [12]:
# --- Linear regression prediction for 2025 (losses and complaints) ---

# We're still using the same years list defined earlier: [2019, 2020, 2021, 2022, 2023, 2024]

# Step 1: recompute total losses/complaints across all years to find top countries.
df_loss['Total_Losses']     = df_loss[[f'{y}_Losses'     for y in years]].sum(axis=1)
df_loss['Total_Complaints'] = df_loss[[f'{y}_Complaints' for y in years]].sum(axis=1)

# Identify the union of top 5 countries by total losses and total complaints.
top5_by_losses     = df_loss.nlargest(5, 'Total_Losses')['Country'].tolist()
top5_by_complaints = df_loss.nlargest(5, 'Total_Complaints')['Country'].tolist()
selected_countries = sorted(set(top5_by_losses + top5_by_complaints))

# Step 2: train a linear regression model per country and predict 2025.
predictions = []
for country in selected_countries:
    # Prepare feature array (years) and target arrays (losses & complaints).
    X = np.array(years).reshape(-1, 1)  # shape (6,1)
    y_losses     = np.array([df_loss.loc[df_loss['Country'] == country, f'{yr}_Losses'].values[0]     for yr in years])
    y_complaints = np.array([df_loss.loc[df_loss['Country'] == country, f'{yr}_Complaints'].values[0] for yr in years])

    # Fit linear models.
    lr_loss = LinearRegression().fit(X, y_losses)
    lr_comp = LinearRegression().fit(X, y_complaints)

    # Predict 2025 values.
    pred_loss_2025 = lr_loss.predict(np.array([[2025]]))[0]
    pred_comp_2025 = lr_comp.predict(np.array([[2025]]))[0]

    predictions.append({
        'Country': country,
        'Year': 2025,
        'Predicted_Losses': pred_loss_2025,
        'Predicted_Complaints': pred_comp_2025
    })

# Convert predictions to a DataFrame and save to CSV (easy import into Power BI).
pred_df_linear = pd.DataFrame(predictions)
pred_df_linear.to_csv('Code_Output/predicted_losses_linear_2025.csv', index=False)

# Step 3: build a tidy table of actual vs. predicted values for each country/year.
rows = []
for country in selected_countries:
    # Actual data for 2019–2024.
    for yr in years:
        rows.append({
            'Country': country,
            'Year': yr,
            'Losses_Billions': df_loss.loc[df_loss['Country'] == country, f'{yr}_Losses'].values[0]     / 1e9,
            'Complaints_Millions': df_loss.loc[df_loss['Country'] == country, f'{yr}_Complaints'].values[0] / 1e6,
            'Type': 'Actual'
        })
    # Predicted 2025 values.
    pred_loss_b  = pred_df_linear.loc[pred_df_linear['Country'] == country, 'Predicted_Losses'].values[0]     / 1e9
    pred_comp_m  = pred_df_linear.loc[pred_df_linear['Country'] == country, 'Predicted_Complaints'].values[0] / 1e6
    rows.append({
        'Country': country,
        'Year': 2025,
        'Losses_Billions': pred_loss_b,
        'Complaints_Millions': pred_comp_m,
        'Type': 'Predicted'
    })

# Create the tidy DataFrame and save it for Power BI.
actual_vs_pred_linear_df = pd.DataFrame(rows)
actual_vs_pred_linear_df.to_csv('Code_Output/actual_vs_predicted_linear_by_country.csv', index=False)

# Optional: display the first few rows for verification.
actual_vs_pred_linear_df.head()

,Country,Year,Losses_Billions,Complaints_Millions,Type
0,DE,2019,1.559416,0.095136,Actual
1,DE,2020,2.432408,0.148495,Actual
2,DE,2021,2.226372,0.164061,Actual
3,DE,2022,3.442138,0.203346,Actual
4,DE,2023,5.575128,0.178259,Actual


## 6. Forecasting European countries for 2025

In [13]:
# --- European-only linear regression prediction with country codes & names ---

# 1. Define dictionaries for EU country codes and for full country names.
eu_country_codes = {
    "PT": "Portugal", "AT": "Austria", "RO": "Romania", "BE": "Belgium",
    "BG": "Bulgaria", "SE": "Sweden", "SI": "Slovenia", "SK": "Slovakia",
    "CY": "Cyprus", "CZ": "Czech Republic", "DE": "Germany", "DK": "Denmark",
    "EE": "Estonia", "ES": "Spain", "FI": "Finland", "FR": "France",
    "GR": "Greece", "HR": "Croatia", "HU": "Hungary", "IE": "Ireland",
    "IT": "Italy", "LT": "Lithuania", "LU": "Luxembourg", "LV": "Latvia",
    "MT": "Malta", "NL": "Netherlands", "PL": "Poland", "EU": "Europe region"
}

country_codes = {
    "PR": "Puerto Rico", "PS": "Palestine", "PT": "Portugal", "PY": "Paraguay",
    "AE": "United Arab Emirates", "AF": "Afghanistan", "AL": "Albania", "AM": "Armenia",
    "AO": "Angola", "AR": "Argentina", "AT": "Austria", "AU": "Australia",
    "AZ": "Azerbaijan", "RO": "Romania", "BA": "Bosnia and Herzegovina", "RS": "Serbia",
    "BD": "Bangladesh", "RU": "Russia", "BE": "Belgium", "BG": "Bulgaria",
    "BH": "Bahrain", "SA": "Saudi Arabia", "BR": "Brazil", "SC": "Seychelles",
    "SE": "Sweden", "SG": "Singapore", "SI": "Slovenia", "BY": "Belarus",
    "SK": "Slovakia", "BZ": "Belize", "CA": "Canada", "SV": "El Salvador",
    "CH": "Switzerland", "SZ": "Eswatini", "CL": "Chile", "CN": "China",
    "CO": "Colombia", "CR": "Costa Rica", "TH": "Thailand", "CY": "Cyprus",
    "CZ": "Czech Republic", "TR": "Turkey", "DE": "Germany", "TW": "Taiwan",
    "TZ": "Tanzania", "DK": "Denmark", "DO": "Dominican Republic", "UA": "Ukraine",
    "UG": "Uganda", "US": "United States", "EC": "Ecuador", "EE": "Estonia",
    "EG": "Egypt", "UZ": "Uzbekistan", "ES": "Spain", "VE": "Venezuela",
    "VG": "British Virgin Islands", "VN": "Vietnam", "FI": "Finland", "FR": "France",
    "GB": "United Kingdom", "GE": "Georgia", "GH": "Ghana", "GN": "Guinea",
    "GR": "Greece", "GT": "Guatemala", "HK": "Hong Kong", "HN": "Honduras",
    "HR": "Croatia", "YE": "Yemen", "HU": "Hungary", "ID": "Indonesia",
    "IE": "Ireland", "IL": "Israel", "IN": "India", "ZA": "South Africa",
    "IQ": "Iraq", "IR": "Iran", "IS": "Iceland", "IT": "Italy",
    "ZW": "Zimbabwe", "JO": "Jordan", "JP": "Japan", "KE": "Kenya",
    "KG": "Kyrgyzstan", "KH": "Cambodia", "KN": "Saint Kitts and Nevis", "KR": "South Korea",
    "KZ": "Kazakhstan", "LB": "Lebanon", "LK": "Sri Lanka", "LT": "Lithuania",
    "LU": "Luxembourg", "LV": "Latvia", "LY": "Libya", "MD": "Moldova",
    "MM": "Myanmar", "MN": "Mongolia", "MT": "Malta", "MV": "Maldives",
    "MX": "Mexico", "MY": "Malaysia", "MZ": "Mozambique", "NG": "Nigeria",
    "NI": "Nicaragua", "NL": "Netherlands", "NO": "Norway", "NP": "Nepal",
    "NZ": "New Zealand", "OM": "Oman", "PA": "Panama", "PE": "Peru",
    "PG": "Papua New Guinea", "PH": "Philippines", "PK": "Pakistan", "PL": "Poland",
    "NaN": "NaN"  # Represents missing or undefined country
}

# 2. Identify which EU country codes appear in our dataset (df_loss). Ignore codes not present.
eu_codes_in_data = [
    code for code in eu_country_codes.keys()
    if code in df_loss['Country'].unique()
]

# 3. Prepare a European subset of df_loss.
df_loss_eu = df_loss[df_loss['Country'].isin(eu_codes_in_data)].copy()

# 3a. Add full country names and a numeric Country ID.
# This makes the data human-friendly and works well with Power BI slicers.
df_loss_eu = df_loss_eu.rename(columns={'Country': 'Country Code'})
df_loss_eu['Country'] = df_loss_eu['Country Code'].map(country_codes)

# Drop any rows where the country code was missing (rare but safe).
df_loss_eu = df_loss_eu.dropna(subset=['Country Code'])

# Assign a unique Country ID (starting at 100, increment by 1).
# Sorting ensures deterministic ordering.
df_loss_eu = df_loss_eu.sort_values('Country Code')
df_loss_eu['Country ID'] = range(101, 101 + len(df_loss_eu))

# 4. Fit linear regression models (2019–2024) and predict 2025 for losses and complaints.
eu_predictions = []
for code in eu_codes_in_data:
    # Use the code to retrieve the full country name and ensure a row exists
    losses = [df_loss_eu.loc[df_loss_eu['Country Code'] == code, f'{yr}_Losses'].values[0] for yr in years]
    comps  = [df_loss_eu.loc[df_loss_eu['Country Code'] == code, f'{yr}_Complaints'].values[0] for yr in years]
    X = np.array(years).reshape(-1, 1)

    # Fit separate linear models for losses and complaints.
    lr_loss = LinearRegression().fit(X, losses)
    lr_comp = LinearRegression().fit(X, comps)

    pred_loss_2025 = lr_loss.predict(np.array([[2025]]))[0]
    pred_comp_2025 = lr_comp.predict(np.array([[2025]]))[0]

    eu_predictions.append({
        'Country Code': code,
        'Country': country_codes.get(code, code),
        'Country ID': df_loss_eu.loc[df_loss_eu['Country Code'] == code, 'Country ID'].values[0],
        'Year': 2025,
        'Predicted_Losses': pred_loss_2025,
        'Predicted_Complaints': pred_comp_2025
    })

pred_df_linear_eu = pd.DataFrame(eu_predictions)
pred_df_linear_eu.to_csv('Code_Output/predicted_losses_linear_2025_europe.csv', index=False)

# 5. Build a tidy table combining actual 2019–2024 data and the 2025 predictions.
eu_rows = []
for code in eu_codes_in_data:
    country_name = country_codes.get(code, code)
    country_id   = df_loss_eu.loc[df_loss_eu['Country Code'] == code, 'Country ID'].values[0]

    # Add actual data rows
    for yr in years:
        eu_rows.append({
            'Country Code': code,
            'Country': country_name,
            'Country ID': country_id,
            'Year': yr,
            'Losses_Billions': df_loss_eu.loc[df_loss_eu['Country Code'] == code, f'{yr}_Losses'].values[0] / 1e9,
            'Complaints_Millions': df_loss_eu.loc[df_loss_eu['Country Code'] == code, f'{yr}_Complaints'].values[0] / 1e6,
            'Type': 'Actual'
        })

    # Add 2025 prediction row
    pred_loss_b = pred_df_linear_eu.loc[pred_df_linear_eu['Country Code'] == code, 'Predicted_Losses'].values[0] / 1e9
    pred_comp_m = pred_df_linear_eu.loc[pred_df_linear_eu['Country Code'] == code, 'Predicted_Complaints'].values[0] / 1e6
    eu_rows.append({
        'Country Code': code,
        'Country': country_name,
        'Country ID': country_id,
        'Year': 2025,
        'Losses_Billions': pred_loss_b,
        'Complaints_Millions': pred_comp_m,
        'Type': 'Predicted'
    })

actual_vs_pred_linear_eu_df = pd.DataFrame(eu_rows)
actual_vs_pred_linear_eu_df.to_csv(
    'Code_Output/actual_vs_predicted_linear_by_country_europe.csv',
    index=False
)

# Optional: display to verify
actual_vs_pred_linear_eu_df.head()

,Country Code,Country,Country ID,Year,Losses_Billions,Complaints_Millions,Type
0,PT,Portugal,123,2019,0.013870,0.001119,Actual
1,PT,Portugal,123,2020,0.012391,0.002020,Actual
2,PT,Portugal,123,2021,0.018206,0.002102,Actual
3,PT,Portugal,123,2022,0.030859,0.001918,Actual
4,PT,Portugal,123,2023,0.028700,0.002178,Actual


## 6. Build a tidy table for actual vs predicted losses

To visualise the results in Power BI, we prepare a **tidy** table where each row represents a single observation (country–year–loss type).

This table includes:

- The actual losses for 2019–2024 (in billions).
- The predicted loss for 2025 (in billions).
- A column named `Type` indicating whether the value is Actual or Predicted.

We save this table to `actual_vs_predicted_losses_by_country.csv`.

In [16]:
pred_df_linear

,Country,Year,Predicted_Losses,Predicted_Complaints
0,DE,2025,6.343586e+09,194659.866667
1,ES,2025,1.564979e+09,94794.533333
2,FR,2025,4.678887e+09,258890.200000
3,IT,2025,1.599628e+09,102435.533333
4,NL,2025,4.027589e+09,103828.200000
5,PL,2025,1.574739e+09,52664.333333
6,SE,2025,1.390069e+09,100690.133333


In [17]:
# 6.1 Build a tidy DataFrame of actual and predicted losses
rows = []
for country in selected_countries:
    # Actual data: convert each year's loss to billions
    actual_losses = [df_loss.loc[df_loss['Country'] == country, f'{year}_Losses'].values[0] for year in years]
    actual_losses_b = [val / 1e9 for val in actual_losses]
    for year_val, val in zip(years, actual_losses_b):
        rows.append({
            'Country': country,
            'Year': year_val,
            'Losses_Billions': val,
            'Type': 'Actual'
        })
    # Predicted: use the regression prediction for 2025
    predicted_b = pred_df_linear.loc[pred_df_linear['Country'] == country, 'Predicted_Losses'].values[0] / 1e9
    rows.append({
        'Country': country,
        'Year': 2025,
        'Losses_Billions': predicted_b,
        'Type': 'Predicted'
    })

# 6.2 Create a DataFrame and save to CSV
pred_chart_df = pd.DataFrame(rows)
pred_chart_df.to_csv('Code_Output/actual_vs_predicted_losses_by_country.csv', index=False)

# 6.3 Display a few rows to verify
pred_chart_df.head()

,Country,Year,Losses_Billions,Type
0,DE,2019,1.559416,Actual
1,DE,2020,2.432408,Actual
2,DE,2021,2.226372,Actual
3,DE,2022,3.442138,Actual
4,DE,2023,5.575128,Actual


## 7. Summary and next steps

We have now:

- Loaded and cleaned the cyber crime dataset.
- Calculated annual totals and changes.
- Identified top countries for complaints and losses.
- Evaluated the correlation between complaints and losses.
- Predicted 2025 losses using the average change.
- Prepared a tidy dataset for plotting actual vs predicted losses.

All tables have been saved to the `Code_Output` folder as CSV files.  \
We can now import these files into Power BI to create bar charts, line charts, and other visualisations.  \
In Power BI remember to set the appropriate data types (e.g., `Year` as a whole number) and choose **Sum** for numeric fields.

# Section 2 - Analysis of cyber attacks

## Cyber Incidents Dataset — Notebook Overview

This notebook uses a curated subset of a Zenodo-hosted catalog of cyber incidents. Each row represents a single incident targeting an entity in a specific country, with attributes describing when it started, who targeted whom, how it was carried out, what was impacted, and whether it may have breached international law. The data supports exploratory analysis of timelines, geographies, attack categories, and outcomes (e.g., political responses, intelligence impacts).

**Column Dictionary**

* **Start date** — The calendar date on which the incident began,
suitable for parsing to datetime.

* **Incident type** — A categorical label describing the specific mode of attack (e.g., DDoS, intrusion, malware, defacement, phishing).

* **Receiver country** — The primary country that was targeted or affected by the incident.

* **Category** — A higher-level grouping of the incident (e.g., espionage, disruption, influence, sabotage) used for broader aggregation.

* **Initiator country** — The country attributed as the origin/sponsor of the incident (may be Unknown or disputed).

* **Political responses** — Recorded official or public reactions to the incident (e.g., sanctions, diplomatic statements, expulsions), often as free text and potentially multi-valued.

* **MITRE Impact** — The effect of the incident aligned to MITRE ATT&CK “Impact” outcomes (e.g., service disruption, data manipulation/destruction, resource hijacking).

* **Intelligence impact** — A qualitative assessment of information compromise or manipulation  

  *Scale 1–5:*

  1 — no data breach/exfiltration, manipulation, or leaking;

  2 — minor data breach/exfiltration (no sensitive/critical information), with no manipulation or leaking;

  3 — data manipulation (e.g., deletion/altering) or a major data breach/exfiltration, but without leaking/manipulation;

  4 — minor data breach/exfiltration with manipulation and/or leaking;

  5 — major data breach/exfiltration of sensitive/critical information and data manipulation and/or leaking.

* **International Law Breach Indicator** — A binary/ordinal flag indicating whether the incident was assessed as breaching international law (e.g., Yes/No or 1/0).

* **Year of incident** — The four-digit year associated with the incident (usually derived from Start date) for convenient time-based grouping.

In [ ]:
#df_threats.head()

### Category vs Intelligence Impact

In [ ]:
impact_counts = df_threats.groupby(["Intelligence impact", "Category"])["Incident type"].count().unstack(fill_value=0)
impact_counts = impact_counts[impact_counts.sum(axis=0).sort_values(ascending=False).index]

x = np.arange(len(impact_counts.index))
width = 0.8 / len(impact_counts.columns)

plt.figure(figsize=(14,7))
for i, category in enumerate(impact_counts.columns):
    plt.bar(x + i*width, impact_counts[category], width=width, label=category)

plt.xticks(x + width*(len(impact_counts.columns)-1)/2, impact_counts.index)
plt.xlabel("Intelligence Impact (1–5)")
plt.ylabel("Number of Incidents")
plt.title("Incidents per Category by Intelligence Impact (sorted globally)")
plt.legend(title="Category", bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()

### Most vulnerable sector (by number of incidents)

In [ ]:
df_threats["Start date"] = pd.to_datetime(df_threats["Start date"], errors="coerce")
df_threats["Year"] = df_threats["Start date"].dt.year

category_counts_per_year = df_threats.groupby(["Category", "Year"])["Incident type"].count().unstack(fill_value=0)

category_counts_per_year = category_counts_per_year.loc[category_counts_per_year.sum(axis=1).sort_values(ascending=False).index]

#new_pl = category_counts_per_year[category_counts_per_year['Year'] > 2021]

plt.figure(figsize=(14,7))
category_counts_per_year.plot(kind="bar", stacked=True, figsize=(14,7), colormap="tab20")
plt.title("Number of Incidents per Year (per Category, sorted by total incidents)")
plt.xlabel("Category")
plt.ylabel("Number of Incidents")
plt.xticks(rotation=45, ha="right")
plt.legend(title="Year", bbox_to_anchor=(1.05, 1), loc="upper left")
plt.tight_layout()
plt.show()

max_per_category = category_counts_per_year.idxmax(axis=1)
counts_per_category = category_counts_per_year.max(axis=1)

### Incident type vs MITRE Impact

In [ ]:
incident_mitre = df_threats.groupby(["Incident type", "MITRE Impact"])["Start date"].count().unstack(fill_value=0)

incident_mitre.plot(kind="bar", stacked=True, figsize=(14,7))
plt.title("Incident Type vs MITRE Impact")
plt.ylabel("Number of Incidents")
plt.xlabel("Incident Type")
plt.xticks(rotation=45, ha="right")
plt.tight_layout()
plt.show()

### Incident type vs Year (time trends)

#### Comment (Hide):

For EU data:

over span of 2017-2024 and span of 2022-2024 the distribution stays the same

However,

for world data: 

over span of 2017-2024 the most common type is Data theft

over span of 2022-2024 the most common type is Disruption

#### Plot

In [ ]:
df_threats["Year"] = pd.to_datetime(df_threats["Start date"], errors="coerce").dt.year

# Group by Incident type and Year
incident_type_year = df_threats.groupby(["Incident type", "Year"])["Start date"].count().unstack(fill_value=0)

# Sort incident types by total number of incidents (descending)
incident_type_year = incident_type_year.loc[incident_type_year.sum(axis=1).sort_values(ascending=False).index]

# ~~~ This line is related to the comment
new_plot = incident_type_year#[[2024, 2023, 2022]]#.transpose()

# Plot stacked bar chart
new_plot.plot(
    kind="bar",
    stacked=True,
    figsize=(14,7),
    colormap="tab20"
)

plt.title("Incidents per Type by Year (Stacked, Descending Order)")
plt.xlabel("Incident Type")
plt.ylabel("Number of Incidents")
plt.xticks(rotation=45, ha="right")
plt.legend(title="Year", bbox_to_anchor=(1.05, 1), loc="upper left")
plt.tight_layout()
plt.show()

# Section 3 - Combining Data sets

### Country Codes Hide Cell

In [ ]:
country_codes = {
    "PR": "Puerto Rico", "PS": "Palestine", "PT": "Portugal", "PY": "Paraguay",
    "AE": "United Arab Emirates", "AF": "Afghanistan", "AL": "Albania", "AM": "Armenia",
    "AO": "Angola", "AR": "Argentina", "AT": "Austria", "AU": "Australia",
    "AZ": "Azerbaijan", "RO": "Romania", "BA": "Bosnia and Herzegovina", "RS": "Serbia",
    "BD": "Bangladesh", "RU": "Russia", "BE": "Belgium", "BG": "Bulgaria",
    "BH": "Bahrain", "SA": "Saudi Arabia", "BR": "Brazil", "SC": "Seychelles",
    "SE": "Sweden", "SG": "Singapore", "SI": "Slovenia", "BY": "Belarus",
    "SK": "Slovakia", "BZ": "Belize", "CA": "Canada", "SV": "El Salvador",
    "CH": "Switzerland", "SZ": "Eswatini", "CL": "Chile", "CN": "China",
    "CO": "Colombia", "CR": "Costa Rica", "TH": "Thailand", "CY": "Cyprus",
    "CZ": "Czech Republic", "TR": "Turkey", "DE": "Germany", "TW": "Taiwan",
    "TZ": "Tanzania", "DK": "Denmark", "DO": "Dominican Republic", "UA": "Ukraine",
    "UG": "Uganda", "US": "United States", "EC": "Ecuador", "EE": "Estonia",
    "EG": "Egypt", "UZ": "Uzbekistan", "ES": "Spain", "VE": "Venezuela",
    "VG": "British Virgin Islands", "VN": "Vietnam", "FI": "Finland", "FR": "France",
    "GB": "United Kingdom", "GE": "Georgia", "GH": "Ghana", "GN": "Guinea",
    "GR": "Greece", "GT": "Guatemala", "HK": "Hong Kong", "HN": "Honduras",
    "HR": "Croatia", "YE": "Yemen", "HU": "Hungary", "ID": "Indonesia",
    "IE": "Ireland", "IL": "Israel", "IN": "India", "ZA": "South Africa",
    "IQ": "Iraq", "IR": "Iran", "IS": "Iceland", "IT": "Italy",
    "ZW": "Zimbabwe", "JO": "Jordan", "JP": "Japan", "KE": "Kenya",
    "KG": "Kyrgyzstan", "KH": "Cambodia", "KN": "Saint Kitts and Nevis", "KR": "South Korea",
    "KZ": "Kazakhstan", "LB": "Lebanon", "LK": "Sri Lanka", "LT": "Lithuania",
    "LU": "Luxembourg", "LV": "Latvia", "LY": "Libya", "MD": "Moldova",
    "MM": "Myanmar", "MN": "Mongolia", "MT": "Malta", "MV": "Maldives",
    "MX": "Mexico", "MY": "Malaysia", "MZ": "Mozambique", "NG": "Nigeria",
    "NI": "Nicaragua", "NL": "Netherlands", "NO": "Norway", "NP": "Nepal",
    "NZ": "New Zealand", "OM": "Oman", "PA": "Panama", "PE": "Peru",
    "PG": "Papua New Guinea", "PH": "Philippines", "PK": "Pakistan", "PL": "Poland",
    'NaN': 'NaN'  # Represents missing or undefined country
}

### DataFrame - Trasform Country Codes to Country Names

In [ ]:
# final output dataFrame for this cell = cntry_loss

cntry_loss = df_net_crime.copy() #copy df
cntry_loss = cntry_loss.rename(columns = {'Country' :'Country Code'}).sort_values(by = 'Country Code') # rename column and sort
#print(cntry_loss.loc[103])
#print(cntry_loss.shape)
cntry_loss = cntry_loss.dropna() # delete the nan columns
#cntry_loss = cntry_loss.drop(axis = 0, index = 103) # delete the nan column
#print(cntry_loss.shape)

column = cntry_loss['Country Code'] # define column variable

# create empty lists
country_col = []
country_id = []
# create a 3 digit country ID
value = 100

for i in column:
    #print(country_codes[i])
    # from list append
    country_col.append(country_codes[i])
    value += 1
    country_id.append((value))

# create new columns
cntry_loss.insert(loc = 1, column = 'Country', value = country_col)
cntry_loss.insert(loc = 1, column = 'Country ID', value = country_id)
# set index = Country
cntry_loss = cntry_loss.set_index('Country').sort_index().copy()

In [ ]:
cntry_loss

### Join DataFrames

In [ ]:
# final output dataFrame for this cell = df_join_23

# work with world data
cntry_wrld = df_world.sort_index()[['Population', 'GDP']].copy() # choose relevant cols
cntry_wrld = cntry_wrld.dropna() # delete the nan columns

# ~~~ make Population into type = float
list_pop = [] # list of population

for i in cntry_wrld['Population']: 
    i = i.replace(',', '') # delete chars
    list_pop.append(float(i)) # append float(population)

cntry_wrld['Population'] = list_pop

list_gdp = []

# ~~~ make GDP into type = float
for i in cntry_wrld['GDP']: 
    i = i.replace(',', '') # delete chars
    i = i.replace('$', '')
    list_gdp.append(float(i))

cntry_wrld['GDP'] = list_gdp

# WORLD data are from 2023
# df.join() is for joining the dfs by indeces 
df_join_23 = cntry_wrld.join(cntry_loss[['2023_Complaints', '2023_Losses']], lsuffix = '_l', how = 'inner')

df_join_23.to_csv('World_Loss_Data.csv')

In [ ]:
df_join_23['2023_Complaints'].max()

### Correlations

In [ ]:
new_df = df_join_23[['2023_Complaints', 'Population']]
new_df.corr(numeric_only = True)

In [ ]:
new_df = df_join_23[['2023_Complaints', 'GDP']]
new_df.corr(numeric_only = True)

## Start Plotting

In [ ]:
arr_com = np.array(df_join_23.sort_values(by = '2023_Complaints')['2023_Complaints'])
arr_pop = np.array(df_join_23.sort_values(by ='2023_Complaints')['Population'])
arr_gdp = np.array(df_join_23.sort_values(by ='2023_Complaints')['GDP'])

In [ ]:
plt.plot(arr_com, arr_gdp, 'o', color = 'g')
plt.xlabel('Number of Cyber Incidents')
plt.ylabel('GDP')

In [ ]:
plt.plot(arr_com, arr_pop, 'o', color = 'r')
plt.xlabel('Number of Cyber Incidents')
plt.ylabel('Population')